In [1]:
### catboost  subit11

In [2]:
###Carga de librerias, lectura del dataset

In [3]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
#Modificar dimensiones predeterminadas de visibilidad de columnas y filasimport warnings
warnings.filterwarnings('ignore')
x = 52
pd.set_option('display.max_columns', x) #Maximo de columnas a ver
pd.set_option('display.max_rows', x) #Maximo de Filas a ver

In [6]:
path_dataset = 'train.csv'
train = pd.read_csv(path_dataset)

In [7]:
path_dataset = 'test.csv'
test = pd.read_csv(path_dataset)

In [8]:
train.head()

,Opportunity_ID,Region,Billing_Country,Account_Type,Opportunity_Type,Account_Owner,Account_Name,Conversión_USD,Delivery_Quarter,TRF,Stage,promedio_preciov_region,promedio_preciov_paises,promedio_preciov_Account,promedio_preciov_Oportunity,promedio_preciov_vendedor,Wilsonxproporcion_region,Wilsonxproporcion_paises,Wilsonxproporcion_vendedor
0,0,EMEA,Netherlands,Account_Type_2,Opportunity_Type_1,Person_Name_51,Account_Name_619,6305214,Q2,10,Closed Lost,246888.239174,0.000000,0.000000,0.000000,0.000000,0.514817,0.513767,0.325851
1,1,EMEA,Netherlands,Account_Type_2,Opportunity_Type_1,Person_Name_51,Account_Name_619,57673,Q1,0,Closed Won,246888.239174,177583.146032,125034.170409,133491.432403,85165.046512,0.514817,0.513767,0.325851
2,2,Americas,United States,Account_Type_5,Opportunity_Type_1,Person_Name_64,Account_Name_1794,83865,Q1,0,Closed Won,422783.847183,432143.813675,958150.887348,133491.432403,75357.668421,0.445687,0.448366,0.616299
3,3,Americas,United States,Account_Type_5,Opportunity_Type_19,Person_Name_8,Account_Name_1201,7421881,Q1,14,Closed Lost,422783.847183,0.000000,0.000000,0.000000,0.000000,0.445687,0.448366,0.561500
4,4,Americas,United States,Account_Type_5,Opportunity_Type_19,Person_Name_8,Account_Name_1201,13357192,Q1,25,Closed Lost,422783.847183,0.000000,0.000000,0.000000,0.000000,0.445687,0.448366,0.561500


In [9]:
train.head()

,Opportunity_ID,Region,Billing_Country,Account_Type,Opportunity_Type,Account_Owner,Account_Name,Conversión_USD,Delivery_Quarter,TRF,Stage,promedio_preciov_region,promedio_preciov_paises,promedio_preciov_Account,promedio_preciov_Oportunity,promedio_preciov_vendedor,Wilsonxproporcion_region,Wilsonxproporcion_paises,Wilsonxproporcion_vendedor
0,0,EMEA,Netherlands,Account_Type_2,Opportunity_Type_1,Person_Name_51,Account_Name_619,6305214,Q2,10,Closed Lost,246888.239174,0.000000,0.000000,0.000000,0.000000,0.514817,0.513767,0.325851
1,1,EMEA,Netherlands,Account_Type_2,Opportunity_Type_1,Person_Name_51,Account_Name_619,57673,Q1,0,Closed Won,246888.239174,177583.146032,125034.170409,133491.432403,85165.046512,0.514817,0.513767,0.325851
2,2,Americas,United States,Account_Type_5,Opportunity_Type_1,Person_Name_64,Account_Name_1794,83865,Q1,0,Closed Won,422783.847183,432143.813675,958150.887348,133491.432403,75357.668421,0.445687,0.448366,0.616299
3,3,Americas,United States,Account_Type_5,Opportunity_Type_19,Person_Name_8,Account_Name_1201,7421881,Q1,14,Closed Lost,422783.847183,0.000000,0.000000,0.000000,0.000000,0.445687,0.448366,0.561500
4,4,Americas,United States,Account_Type_5,Opportunity_Type_19,Person_Name_8,Account_Name_1201,13357192,Q1,25,Closed Lost,422783.847183,0.000000,0.000000,0.000000,0.000000,0.445687,0.448366,0.561500


In [10]:
###Preprocesamiento

In [11]:
train['Stage']= (train['Stage'] == 'Closed Won') +0

In [12]:
for column in train.columns:
    if train[column].dtype == 'int32':
        train[column] = train[column].astype('float64')
print(train.dtypes)

Opportunity_ID                   int64
Region                          object
Billing_Country                 object
Account_Type                    object
Opportunity_Type                object
Account_Owner                   object
Account_Name                    object
Conversión_USD                   int64
Delivery_Quarter                object
TRF                              int64
Stage                          float64
promedio_preciov_region        float64
promedio_preciov_paises        float64
promedio_preciov_Account       float64
promedio_preciov_Oportunity    float64
promedio_preciov_vendedor      float64
Wilsonxproporcion_region       float64
Wilsonxproporcion_paises       float64
Wilsonxproporcion_vendedor     float64
dtype: object


In [13]:
### Target meaning encoding

In [14]:
# Import KFold
from sklearn.model_selection import KFold

In [15]:
def test_mean_target_encoding(train, test, target, categorical, alpha=5):
    # Calculate global mean on the train data
    global_mean = train[target].mean()
    
    # Group by the categorical feature and calculate its properties
    train_groups = train.groupby(categorical)
    category_sum = train_groups[target].sum()
    category_size = train_groups.size()
    
    # Calculate smoothed mean target statistics
    train_statistics = (category_sum + global_mean * alpha) / (category_size + alpha)
   
    # Apply statistics to the test data and fill new categories
    test_feature = test[categorical].map(train_statistics).fillna(global_mean)
    return test_feature.values

In [16]:
def train_mean_target_encoding(train, target, categorical, alpha=5):
    # Create 5-fold cross-validation
    kf = KFold(n_splits=5, random_state=123, shuffle=True)
    train_feature = pd.Series(index=train.index)
    
    # For each folds split
    for train_index, test_index in kf.split(train):
        cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
      
        # Calculate out-of-fold statistics and apply to cv_test
        cv_test_feature = test_mean_target_encoding(cv_train, cv_test, target, categorical, alpha)
        
        # Save new feature for this particular fold
        train_feature.iloc[test_index] = cv_test_feature       
    return train_feature.values

In [17]:
def mean_target_encoding(train, test, target, categorical, alpha=5):
  
    # Get the train feature
    train_feature = train_mean_target_encoding(train, target, categorical, alpha)
  
    # Get the test feature
    test_feature = test_mean_target_encoding(train, test, target, categorical, alpha)
    
    # Return new features to add to the model
    return train_feature, test_feature

In [18]:
columns = ['Region', 'Billing_Country', 'Account_Type','Opportunity_Type', 'Account_Owner', 'Account_Name','Delivery_Quarter']
for column in columns:     
    train[column], test[column] = mean_target_encoding(train=train,
                                                                test=test,
                                                                target='Stage',
                                                                categorical=column,
                                                                alpha=10)

In [19]:
###Seleccion de Features

In [20]:
train.columns

Index(['Opportunity_ID', 'Region', 'Billing_Country', 'Account_Type',
       'Opportunity_Type', 'Account_Owner', 'Account_Name', 'Conversión_USD',
       'Delivery_Quarter', 'TRF', 'Stage', 'promedio_preciov_region',
       'promedio_preciov_paises', 'promedio_preciov_Account',
       'promedio_preciov_Oportunity', 'promedio_preciov_vendedor',
       'Wilsonxproporcion_region', 'Wilsonxproporcion_paises',
       'Wilsonxproporcion_vendedor'],
      dtype='object')

In [21]:
features = ['Region', 'Billing_Country', 'Account_Type',
       'Opportunity_Type', 'Account_Owner', 'Account_Name', 'Conversión_USD',
       'Delivery_Quarter', 'TRF', 'Wilsonxproporcion_region', 'Wilsonxproporcion_paises',
       'Wilsonxproporcion_vendedor']

In [22]:
### Modelos

In [23]:
X = train.drop(['Stage'], axis=1)
y = train['Stage']

In [24]:
###Separando en conjuntos de entrenamiento y de testeo¶

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [26]:
###Entrenando los modelos

In [27]:
from catboost import CatBoostClassifier

In [28]:
lgb_clf = CatBoostClassifier()

In [ ]:
### permite buenos resultados con parametros por defecto

In [29]:
lgb_clf.fit(X_train[features], y_train)

Learning rate set to 0.031355
0:	learn: 0.6792245	total: 94.6ms	remaining: 1m 34s
1:	learn: 0.6654303	total: 104ms	remaining: 51.7s
2:	learn: 0.6533679	total: 113ms	remaining: 37.6s
3:	learn: 0.6419116	total: 126ms	remaining: 31.3s
4:	learn: 0.6312004	total: 137ms	remaining: 27.2s
5:	learn: 0.6208198	total: 148ms	remaining: 24.5s
6:	learn: 0.6121491	total: 158ms	remaining: 22.4s
7:	learn: 0.6033232	total: 167ms	remaining: 20.7s
8:	learn: 0.5944671	total: 177ms	remaining: 19.5s
9:	learn: 0.5864723	total: 187ms	remaining: 18.5s
10:	learn: 0.5797413	total: 202ms	remaining: 18.2s
11:	learn: 0.5738315	total: 212ms	remaining: 17.4s
12:	learn: 0.5678955	total: 221ms	remaining: 16.8s
13:	learn: 0.5615903	total: 231ms	remaining: 16.3s
14:	learn: 0.5562223	total: 242ms	remaining: 15.9s
15:	learn: 0.5511299	total: 256ms	remaining: 15.8s
16:	learn: 0.5465461	total: 266ms	remaining: 15.4s
17:	learn: 0.5421733	total: 276ms	remaining: 15.1s
18:	learn: 0.5384492	total: 288ms	remaining: 14.8s
19:	learn

In [30]:
y_pred=lgb_clf.predict_proba(X_test[features])

In [32]:
# Import log_loss from sklearn
from sklearn.metrics import log_loss

In [33]:
print('Sklearn LogLoss: {:.5f}'.format(log_loss(y_test[features], y_pred)))

KeyError: "None of [Index(['Region', 'Billing_Country', 'Account_Type', 'Opportunity_Type',\n       'Account_Owner', 'Account_Name', 'Conversión_USD', 'Delivery_Quarter',\n       'TRF', 'Wilsonxproporcion_region', 'Wilsonxproporcion_paises',\n       'Wilsonxproporcion_vendedor'],\n      dtype='object')] are in the [index]"

In [35]:
preds_test=lgb_clf.predict_proba(test[features])

In [36]:
preds_test=preds_test[:, 1]

In [37]:
test.head()

,Opportunity_ID,Region,Billing_Country,Account_Type,Opportunity_Type,Account_Owner,Account_Name,Conversión_USD,Delivery_Quarter,TRF,promedio_preciov_region,promedio_preciov_paises,promedio_preciov_Account,promedio_preciov_Oportunity,promedio_preciov_vendedor,Wilsonxproporcion_region,Wilsonxproporcion_paises,Wilsonxproporcion_vendedor
0,10689,0.530441,0.568074,0.706639,0.701381,0.672134,0.501738,194006,0.566725,0,246888.239174,325954.165775,116533.133558,87111.379883,278450.516274,0.514817,0.445687,0.639475
1,10689,0.530441,0.568074,0.706639,0.701381,0.672134,0.501738,94018,0.566725,0,246888.239174,325954.165775,116533.133558,87111.379883,278450.516274,0.514817,0.445687,0.639475
2,10689,0.530441,0.568074,0.706639,0.701381,0.672134,0.501738,151334,0.566725,0,246888.239174,325954.165775,116533.133558,87111.379883,278450.516274,0.514817,0.445687,0.639475
3,10690,0.530441,0.568074,0.706639,0.701381,0.672134,0.501738,291009,0.566725,1,246888.239174,325954.165775,116533.133558,87111.379883,278450.516274,0.514817,0.445687,0.639475
4,10690,0.530441,0.568074,0.706639,0.701381,0.672134,0.501738,188037,0.566725,0,246888.239174,325954.165775,116533.133558,87111.379883,278450.516274,0.514817,0.445687,0.639475


In [38]:
test['Target']=preds_test

In [39]:
test['Target'].shape

(2551,)

In [40]:
sub= test[['Opportunity_ID','Target']]

In [41]:
sub1=sub.groupby(['Opportunity_ID'], as_index=False).mean()

In [43]:
sub1[['Opportunity_ID', 'Target']].to_csv('submission11.csv', index=False)

In [44]:
est = test.drop(['Target'], axis=1)

In [ ]:
##Fin de submision 11